In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.applications import ResNet50V2
from keras.datasets import cifar100
from keras import Sequential
from keras import Input
from keras.layers import UpSampling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import RandomFlip
from keras.layers import RandomTranslation
from keras.layers import RandomRotation
from keras.layers import RandomBrightness
from keras.layers import RandomContrast
from keras.layers import RandomZoom
from keras_cv.layers import RandomShear
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
import re
import os
import datetime

Using TensorFlow backend


In [2]:
n_epoch = 1000 # Il y a le early stopping
batch_size = 100

In [3]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()
X_train = np.float64(X_train)/255
X_test = np.float64(X_test)/255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = 100

In [4]:
resnet = ResNet50V2(include_top=False, weights='imagenet', pooling="avg")

for layer in resnet.layers[:]:
  if (re.match("^(.*(_3_conv|_bn)|(conv3_.*)|post_bn)$", layer.name)):
    layer.trainable = True
  else:
    layer.trainable = False

resnet.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, None, None, 64)       0         ['conv1_conv[0][0]']          
                                                                                         

In [5]:
model = Sequential([
    UpSampling2D(size=(7, 7)),
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomShear(0.2),
    RandomBrightness(0.2,value_range=(0,1)),
    RandomContrast(0.2),
    resnet,
    Dense(256, activation="sigmoid"),
    Dropout(0.25),
    Dense(num_classes, activation="softmax")])

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history_1 = model.fit(X_train, y_train, epochs=n_epoch, batch_size=batch_size, validation_split = 0.1,callbacks = early_stopping_callback)
model.save("/root/modele.keras")

Epoch 1/1000
450/450 [==============================] - 81s 110ms/step - loss: 2.6751 - accuracy: 0.3424 - val_loss: 1.9104 - val_accuracy: 0.4874
Epoch 2/1000
450/450 [==============================] - 47s 105ms/step - loss: 1.8965 - accuracy: 0.4935 - val_loss: 1.6429 - val_accuracy: 0.5460
Epoch 3/1000
450/450 [==============================] - 47s 105ms/step - loss: 1.6494 - accuracy: 0.5550 - val_loss: 1.3630 - val_accuracy: 0.6166
Epoch 4/1000
450/450 [==============================] - 47s 105ms/step - loss: 1.4926 - accuracy: 0.5902 - val_loss: 1.2941 - val_accuracy: 0.6280
Epoch 5/1000
450/450 [==============================] - 47s 105ms/step - loss: 1.4086 - accuracy: 0.6092 - val_loss: 1.2024 - val_accuracy: 0.6484
Epoch 6/1000
450/450 [==============================] - 47s 105ms/step - loss: 1.3228 - accuracy: 0.6319 - val_loss: 1.0996 - val_accuracy: 0.6826
Epoch 7/1000
450/450 [==============================] - 47s 105ms/step - loss: 1.2484 - accuracy: 0.6514 - val_loss: 1

In [7]:
tf.keras.models.load_model("/root/tiers")

OSError: No file or directory found at /root/tiers